In [1]:
!pip install requests beautifulsoup4 selenium -q
!apt update -q
!apt install chromium-chromedriver -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 https://ppa.launchpadc

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import requests

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
store_url = "https://nhathuoclongchau.com.vn/"

data = pd.read_csv("/content/drive/MyDrive/Chatbot/drug_info.csv")

data.head()

,Name,Link
0,Thuốc Actadol 500 Medipharco điều trị các chứn...,/thuoc/actadol-500mg-medipharco-10x10-33851.html
1,Thuốc Allerphast 180mg Mebiphar điều trị viêm ...,/thuoc/allerphast-180mg-4805.html
2,Thuốc A.T Zinc 10mg An Thiên điều trị tiêu chả...,/thuoc/at-zinc-an-thien-10mg-10x10-31306.html
3,Dung dịch uống AtiLude 250mg/5ml An Thiên tiêu...,/thuoc/atilude-250-mg-5-ml-an-thien-6-x5-ong-5...
4,Thuốc Acefalgan 500mg Euvipharm giảm đau nhẹ đ...,/thuoc/acefalgan-500mg-euvipharm-10x10-30343.html


In [4]:
example = store_url + data["Link"][3]

# Gửi yêu cầu đến trang web
response = requests.get(example)
# Phân tích nội dung trang web
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
def get_list_price(url: str, range_list: int, options: webdriver.chrome.options.Options):
  dr = webdriver.Chrome(options=options)

  dr.get(url) # Website used for scrapin

  # list of prices
  list_ = []

  for i in range(1, range_list + 1):
    a = dr.find_element(By.XPATH, f'//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[1]/div[2]/div/div[5]/table/tbody/tr[1]/td[2]/div/span[{i}]/p')
    a.click()

    http = dr.page_source ## get http file
    parse = BeautifulSoup(http, 'html.parser') # parse http file

    # Get Price
    f = parse.find('div', class_="css-7zmm2q").find_all('span')
    text = f[0].text
    text += f[1].text.replace('\xa0', '')

    # Append Price to list_
    list_.append(text)

  print(list_)

  #quitting the browser
  dr.quit()

  return list_

In [6]:
name: str = soup.find_all('h1', class_="css-18o6y07 text-gray-10 inline align-middle font-medium")[0].get_text()

brand: str = soup.find_all('span', class_="text-body2 md:text-label1")[1].a.get_text()

img_link: str = soup.find_all('picture', class_="h-full w-full object-contain gallery-img slide-active")[1].source['srcset']

price = ["None"]

meta_data = {}

for item in soup.find_all('tr', class_ = "content-container"):
  d = item.find_all('td')
  assert len(d) == 2 , "Len does not equal."

  try:
    check = d[0]['class']
    del check
    range_list = len(d[1].div.find_all('span'))
    price = get_list_price(example, range_list, options)
    continue

  except:
    key = d[0].p.text
    try:
      value = d[1].div.text
    except:
      value = d[1].a.p.text
    meta_data[key] = value

['90.000đ/Hộp', '15.000đ/Vỉ', '3.000đ/Ống']


In [7]:
print(name)
print(brand)
print(img_link)
print(price)
print(meta_data)

Dung dịch uống AtiLude 250mg/5ml An Thiên tiêu nhầy, rối loạn đường hô hấp (6 vỉ x 5 ống x 5ml)
An Thiên
https://cdn.nhathuoclongchau.com.vn/unsafe/373x0/filters:quality(90)/https://cms-prod.s3-sgn09.fptcloud.com/DSC_04350_5dfca09de8.jpg
['90.000đ/Hộp', '15.000đ/Vỉ', '3.000đ/Ống']
{'Danh mục': 'Thuốc ho & cảm', 'Dạng bào chế': 'Dung dịch uống', 'Quy cách': 'Hộp 6 Vỉ x 5 Ống', 'Thành phần': 'Carbocisteine', 'Nhà sản xuất': 'AN THIÊN', 'Nước sản xuất': 'Việt Nam', 'Xuất xứ thương hiệu': 'Việt Nam', 'Số đăng ký': 'VD-29690-18', 'Thuốc cần kê toa': 'Không', 'Mô tả ngắn': 'Dung dịch uống AtiLude là sản phẩm của Công ty Cổ phần Dược phẩm An Thiên chứa hoạt chất Carbocisteine. Đây là thuốc dùng để làm tiêu nhầy, dùng trong điều trị bổ trợ các rối loạn đường hô hấp có kèm theo tăng tiết chất nhầy nhớt quá mức, bao gồm chứng khó thở đường hô hấp mạn tính.'}


In [104]:
import time

# names = []
# brands = []
# img_links = []
# prices = []
# metadata = []

for idx, row in data.iterrows():
  if idx < 4801:
    continue
  print("DRUG IDX: ", idx)

  url = store_url + row['Link']

  response = requests.get(url)

  soup = BeautifulSoup(response.content, 'html.parser')

  name = soup.find('h1', class_="css-18o6y07 text-gray-10 inline align-middle font-medium").get_text()

  try:
    brand = soup.find_all('span', class_="text-body2 md:text-label1")[1].a.get_text()
  except:
    try:
      brand = soup.find_all('span', class_="text-body2 md:text-label1")[1].get_text()
    except:
      brand = "None"

  img_link = soup.find('picture', class_="h-full w-full object-contain gallery-img slide-active").source['srcset']

  price = ["None"]

  temp_metadata = {}

  for item in soup.find_all('tr', class_ = "content-container"):
    d = item.find_all('td')
    assert len(d) == 2 , "Len does not equal."

    try:
      check = d[0]['class']
      del check
      range_list = len(d[1].div.find_all('span'))
      price = get_list_price(url, range_list, options)
      continue

    except KeyError:
      key = d[0].p.text
      try:
        value = d[1].div.text
      except:
        try:
          value = d[1].a.p.text
        except:
          value = d[1].text

      temp_metadata[key] = value

  ## Append to list
  names.append(name)
  brands.append(brand)
  img_links.append(img_link)
  prices.append(price)
  metadata.append(temp_metadata)

  if idx % 100 == 0:
    time.sleep(10)

DRUG IDX:  4801
DRUG IDX:  4802
DRUG IDX:  4803
DRUG IDX:  4804
DRUG IDX:  4805
DRUG IDX:  4806
DRUG IDX:  4807
DRUG IDX:  4808
DRUG IDX:  4809
DRUG IDX:  4810
DRUG IDX:  4811
DRUG IDX:  4812
DRUG IDX:  4813
DRUG IDX:  4814
DRUG IDX:  4815
DRUG IDX:  4816
DRUG IDX:  4817
DRUG IDX:  4818
DRUG IDX:  4819
DRUG IDX:  4820
DRUG IDX:  4821
DRUG IDX:  4822
DRUG IDX:  4823
DRUG IDX:  4824
DRUG IDX:  4825
DRUG IDX:  4826
DRUG IDX:  4827
DRUG IDX:  4828
DRUG IDX:  4829
DRUG IDX:  4830
DRUG IDX:  4831
DRUG IDX:  4832
DRUG IDX:  4833
DRUG IDX:  4834
DRUG IDX:  4835
DRUG IDX:  4836
DRUG IDX:  4837
DRUG IDX:  4838
DRUG IDX:  4839
DRUG IDX:  4840
DRUG IDX:  4841
DRUG IDX:  4842
DRUG IDX:  4843
DRUG IDX:  4844
DRUG IDX:  4845
DRUG IDX:  4846
DRUG IDX:  4847
DRUG IDX:  4848
DRUG IDX:  4849
DRUG IDX:  4850
DRUG IDX:  4851
DRUG IDX:  4852
DRUG IDX:  4853
DRUG IDX:  4854
DRUG IDX:  4855
DRUG IDX:  4856
DRUG IDX:  4857
DRUG IDX:  4858
DRUG IDX:  4859
DRUG IDX:  4860
DRUG IDX:  4861
DRUG IDX:  4862
DRUG IDX

In [103]:
data['Link'][4801]

'/thuoc/vesicare-5mg-1802.html'

In [105]:
# Create data frame
df = pd.DataFrame({"Name": names,
                   "Brand": brands,
                   "Image Link": img_links,
                   "Price": prices,
                   "Meatadata": metadata})
df.head()

,Name,Brand,Image Link,Price,Meatadata
0,Thuốc Remem Probiotec điều trị suy tuần hoàn n...,MBC,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,"[536.000đ/Hộp, 134.000đ/Vỉ, 8.933đ/Viên]",{'Danh mục': 'Thuốc hướng thần kinh & thuốc bổ...
1,Dung dịch Rectiofar Pharmedic điều trị táo bón...,Pharmedic,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,"[112.000đ/Hộp, 2.800đ/Ống]","{'Danh mục': 'Thuốc nhuận trường, thuốc xổ', '..."
2,Thuốc Rotorlip 20 DHG điều trị tăng cholestero...,Dhg,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],"{'Danh mục': 'Hệ tim mạch & tạo máu', 'Dạng bà..."
3,Dung dịch Rectiofar Pharmedic điều trị táo bón...,Pharmedic,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,"[125.000đ/Hộp, 2.500đ/Ống]","{'Danh mục': 'Thuốc nhuận trường, thuốc xổ', '..."
4,Thuốc Reduze Aus Biopharm giảm viêm khớp gối n...,Probiotec,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,"[480.000đ/Hộp, 96.000đ/Vỉ, 8.000đ/Viên]",{'Danh mục': 'Các thuốc khác tác động lên hệ c...


In [106]:
df.tail()

,Name,Brand,Image Link,Price,Meatadata
1260,Siro Zincviet Atco hương vani điều trị bệnh ti...,Atco,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],"{'Danh mục': 'Vitamin & khoáng chất', 'Dạng bà..."
1261,Thuốc Zentobiso 5mg Niche Generics điều trị su...,Niche Generics,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],"{'Danh mục': 'Thuốc chẹn thụ thể bêta', 'Dạng ..."
1262,Bột pha hỗn dịch uống Ziptum Sachet 300mg Maxi...,Maxim,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],"{'Danh mục': 'Cephalosporin', 'Dạng bào chế': ..."
1263,Thuốc Zentogout-40 DaviPharm điều trị gout (2 ...,Davipharm,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],{'Danh mục': 'Thuốc trị tăng acid uric máu & b...
1264,Thuốc Zonafil Hà Tây điều trị và dự phòng tái ...,Hà Tây,https://cdn.nhathuoclongchau.com.vn/unsafe/373...,[None],"{'Danh mục': 'Thuốc kháng acid, chống trào ngư..."


In [107]:
# convert to csv
df.to_csv('/content/drive/MyDrive/Chatbot/drug_metadata_3793-5058.csv', index=False)